In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-12-16 22:54:28.359655


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Loading /Volumes/Music/Discog/artists-db/52-DB.p... Found 6169 Artists
Loading data from /Volumes/Music/Discog/diagnostic/albumKnownIDs.p
  --> This file is 3.1MB.
Loading /Volumes/Music/Discog/diagnostic/albumKnownIDs.p

 ============================== Artist ID --- 1000552 ==============================
   3 / 3    	 99593 	 1000552 	 Pieter De Graaf
	 Getting 2 media IDs
	 Gett

  Found album code 6774369 with 2 artists
  Saving /Volumes/Music/Discog/albums/13/712113/6774369.p
  --> This file is 15.7kB.
  Saving /Volumes/Music/Discog/albums/52/1012052/6774369.p
  --> This file is 15.7kB.
  Done and sleeping...

 ============================== Artist ID --- 1012452 ==============================
   1 / 6    	 99594 	 1012452 	 Xibalba (2)
	 Getting 3 media IDs
  Found album code 119277 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1012452/119277.p
  --> This file is 17.3kB.
  Done and sleeping...
	 Getting 1 media IDs
  Found album code 986915 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1012452/986915.p
  --> This file is 16.9kB.
  Done and sleeping...
	 Getting 1 media IDs
	 Getting 1 media IDs
  Found album code 1692484 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1012452/1692484.p
  --> This file is 15.3kB.
  Done and sleeping...

 ============================== Artist ID --- 1013552 ==============================
   0 / 11  

  Found album code 3157993 with 1 artists
  Exists /Volumes/Music/Discog/albums/44/2451944/3157993.p
  Done and sleeping...
	 Getting 9 media IDs
  Found album code 392861 with 1 artists
  Exists /Volumes/Music/Discog/albums/98/1417098/392861.p
  Done and sleeping...
	 Getting 1 media IDs
  Found album code 3873722 with 1 artists
  Saving /Volumes/Music/Discog/albums/42/2923142/3873722.p
  --> This file is 15.8kB.
  Done and sleeping...
	 Getting 68 media IDs
	  Already known and previously downloaded.
	 Getting 6 media IDs
  Found album code 2003347 with 1 artists
  Saving /Volumes/Music/Discog/albums/64/1624964/2003347.p
  --> This file is 16.4kB.
  Done and sleeping...
	 Getting 6 media IDs
  Found album code 1387392 with 2 artists
  Saving /Volumes/Music/Discog/albums/28/728828/1387392.p
  --> This file is 19.3kB.
  Saving /Volumes/Music/Discog/albums/79/397479/1387392.p
  --> This file is 19.3kB.
  Done and sleeping...
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

	 Getting 2 media IDs
  Found album code 8241252 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1026052/8241252.p
  --> This file is 14.6kB.
  Done and sleeping...
	 Getting 1 media IDs
  Found album code 5176731 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1026052/5176731.p
  --> This file is 14.5kB.
  Done and sleeping...

 ============================== Artist ID --- 102752 ==============================
   0 / 6    	 99598 	 102752 	 Future World
	 Getting 2 media IDs
  Found album code 2192190 with 1 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/2192190.p
  --> This file is 15.3kB.
  Done and sleeping...
	 Getting 2 media IDs
  Found album code 1120860 with 1 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/1120860.p
  --> This file is 17.4kB.
  Done and sleeping...
	 Getting 2 media IDs
  Found album code 7900207 with 1 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/7900207.p
  --> This file is 17.2kB.
  Done and sleeping...

== Download Ra

  Found album code 12009451 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1036952/12009451.p
  --> This file is 14.8kB.
  Done and sleeping...
	 Getting 7 media IDs
	 Getting 13 media IDs
  Found album code 11388825 with 2 artists
  Saving /Volumes/Music/Discog/albums/52/1036952/11388825.p
  --> This file is 16.3kB.
  Saving /Volumes/Music/Discog/albums/8/3469608/11388825.p
  --> This file is 16.3kB.
  Done and sleeping...

== Download Rate: 75 / 3.8 = 20.0


 ============================== Artist ID --- 1037052 ==============================
   3 / 8    	 99598 	 1037052 	 Takács Paula
	 Getting 6 media IDs
	 Getting 2 media IDs
  Found album code 13755746 with 3 artists
  Saving /Volumes/Music/Discog/albums/27/192327/13755746.p
  --> This file is 16.2kB.
  Saving /Volumes/Music/Discog/albums/52/1037052/13755746.p
  --> This file is 16.2kB.
  Saving /Volumes/Music/Discog/albums/77/997277/13755746.p
  --> This file is 16.2kB.
  Done and sleeping...

 =========================

  Found album code 626939 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/1054252/626939.p
  --> This file is 16.7kB.
  Done and sleeping...
	 Getting 19 media IDs
	 Getting 15 media IDs


# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)